In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv
import traceback
import os

In [2]:
# 保管先フォルダの名称
OUTPUT_DIR = 'livedoor_news'
# ニュースアクセスランキング（ライブドアニュース）のURL
URL = 'http://news.livedoor.com/ranking/'
# ニュースID抽出用の正規表現（名前付きグループ）
REG_URL = r'(?P<L1>(https?://[^/]+/))(?P<L2>([^/]+))/(?P<L3>([^/]+))/(?P<L4>([^/]+))/'
# UserAgent
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'

In [3]:
def create_folder():
    if not os.path.exists(OUTPUT_DIR):
        # フォルダが存在しなければ、フォルダを作成
        os.makedirs(OUTPUT_DIR)

In [4]:
def get_links():
    try:
        # HTTPリクエストヘッダーにUser Agentを設定
        req = Request(URL, data=None, headers={'User-Agent': USER_AGENT})
        with urlopen(URL) as res:
            # HTMLドキュメントからBeautifulSoupを初期化
            soup = BeautifulSoup(res.read().decode('euc_jp', 'ignore'), 'html.parser')
            # ニュースアクセスランキング部分を検索し、全て（20件）を取得
            soupNewsRanking = soup.find('ol', class_='articleList withRanking').find_all('li')

            articles = []
            for idx, soupNews in enumerate(soupNewsRanking):
                # 3行要約ページURLをHTMLタグの属性から抽出
                url = soupNews.a.get('href')
                # NewsBodyを検索し取得
                soupNewsBody = soupNews.find('div', class_='articleListBody')
                # NewsBodyから各種データを抽出
                article = {
                    'url': url,
                    # ニュースIDを3行要約ページURLから抽出
                    'id': re.search(REG_URL, url).groupdict()['L4'],
                    # タイトル／サマリ／提供元／公開日時をHTMLタグの本文から抽出
                    'title': soupNewsBody.find('h3', class_='articleListTtl').text,
                    'summary': soupNewsBody.find('p', class_='articleListSummary').text,
                    'vender': soupNewsBody.find('p', class_='articleListVender').text,
                    'datetime': soupNewsBody.find('time', class_='articleListDate').text
                }
                articles.append(article)
                print('%2d: %s' %(idx + 1, soupNewsBody.find('h3', class_='articleListTtl').text))

            df = pd.DataFrame(articles)
            # DataFrameからCSVファイルを生成
            # encoding='sjis'だとJupyterLab（CSVTable）上で表示不可なことに注意
            df.to_csv('%s/livedoor_news_ranking.csv' %OUTPUT_DIR, encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)
    except Exception as e:
        # エラー概要
        print('Exception: ', e)
        print('=====')
        # エラー詳細（スタックトレース）
        print(traceback.format_exc().rstrip())
        print('=====')

In [5]:
create_folder()
get_links()

 1: インフルエンザ「今季は大流行の恐れ」 日本感染症学会は接種を呼びかけ
 2: 小泉進次郎氏を涙の退庁に追いやる？「トンデモ失策」の代償
 3: 健康理由に魚食べるヴィーガンの米女性号泣「お魚さん、ごめんなさい」
 4: 覆面パトカーを運転中の警官、スピード違反で青切符を交付される
 5: 深田恭子がFNS歌謡祭で「異変」「復帰早すぎた」と体調気遣う声
 6: 北海道大学内のトイレが外から「丸見え」に「配慮足りなかった」
 7: 橋本聖子氏の「真夏の五輪は無理」発言 東国原英夫氏が「牛乳を吹いた」
 8: 京都大学の教授、乗用車にはねられ死亡 信号機のない交差点で
 9: すぎやまこういちさんが敗血症性ショックで死去「ドラクエ」音楽を担当
10: 眞子さまの結婚＆皇籍離脱で…佳子さまら女性皇族に「別の懸念」
11: 妻に10年間無職を隠して結婚詐欺、夫を逮捕 被害者への謝罪文は5行
12: 山本太郎氏、議員バッジを失い嘆いていた？衆院選は確実に勝ちに行くか
13: 日本の感染者大幅減、韓国で疑問視する声「数字が改ざんされた」
14: へずまりゅう氏、演説で「ゴメンなさいでした」報道陣や聴衆は呆然
15: 「THE TIME,」で中継先のCBCアナが池に転落 ずぶ濡れで生中継
16: 大倉忠義が吉高由里子との関係を匂わせ？うっかりミスで投稿か
17: デヴィ夫人が中国で炎上「芸能界追放」俳優との写真拡散に「無礼」
18: 篠原涼子にピンチ？キャスティング会議で候補に挙がらず…TV局が自主規制か
19: SUBARUがレガシィの新型モデルを発表 7年ぶりの全面改良
20: すぎやまこういち氏が死去 心がけた「聴き減りしない」作曲


In [6]:
# CSVファイルからDataFrameを生成
df = pd.read_csv('livedoor_news/livedoor_news_ranking.csv', encoding='utf-8')
df.head()

,url,id,title,summary,vender,datetime
0,https://news.livedoor.com/topics/detail/20988126/,20988126,インフルエンザ「今季は大流行の恐れ」 日本感染症学会は接種を呼びかけ,昨季はコロナ禍で感染予防対策が徹底され、インフルエンザ患者が激減した,読売新聞オンライン,2021年10月7日 11時56分
1,https://news.livedoor.com/topics/detail/20984857/,20984857,小泉進次郎氏を涙の退庁に追いやる？「トンデモ失策」の代償,進次郎氏を巡っては、「レジ袋政策」に改めて批判が集まっていると政治記者,Smart FLASH,2021年10月6日 9時56分
2,https://news.livedoor.com/topics/detail/20986620/,20986620,健康理由に魚食べるヴィーガンの米女性号泣「お魚さん、ごめんなさい」,「お魚さん、本当にごめんなさい」と泣きながら魚を食べる動画をSNSで公開,Techinsight,2021年10月7日 6時0分
3,https://news.livedoor.com/topics/detail/20986935/,20986935,覆面パトカーを運転中の警官、スピード違反で青切符を交付される,甲州市の国道20号を、制限速度の時速60キロを超えるスピードで走行,読売新聞オンライン,2021年10月7日 7時14分
4,https://news.livedoor.com/topics/detail/20985447/,20985447,深田恭子がFNS歌謡祭で「異変」「復帰早すぎた」と体調気遣う声,万全に見えない深田の様子が放送されると、Twitterでは心配の声が浮上,東スポWeb,2021年10月6日 20時41分
